In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import sys
PROJECT_ROOT = os.path.abspath(os.curdir) ## Please make sure it outputs the correct root, No universal solution for Colab - Jupyter - Local environment
PROJECT_ROOT += "/drive/MyDrive/GCN/" 
print('The Project is running from : ', PROJECT_ROOT)
sys.path.append(PROJECT_ROOT)

The Project is running from :  /content/drive/MyDrive/GCN/


In [3]:
from __future__ import print_function, absolute_import, division

import os
import time
import datetime
import argparse
import numpy as np
import os.path as path
import matplotlib.pyplot as plt


import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader
import pickle

from progress.bar import Bar
from common.log import Logger, savefig
from common.utils import AverageMeter, lr_decay, save_ckpt
from common.graph_utils import adj_mx_from_skeleton
from common.data_utils import fetch, read_3d_data, create_2d_data
from common.generators import PoseGenerator
from common.loss import mpjpe, p_mpjpe
from models.sem_gcn import SemGCN

In [4]:
def pickle_read(filename):
        with open(filename, "rb") as f:
                data = pickle.load(f)
        return data
def dictionary_to_2D_sample(dict_pose):
        output = np.zeros((1,16,2))
        useful_joints=[8,12,13,14,9,10,11,1,1,0,2,3,4,5,6,7]
        for i,e in enumerate(useful_joints):
            output[0][i] = dict_pose[e]
        return output

def dictionary_to_3D_sample(dict_pose):
        output = np.zeros((1,16,3))
        useful_joints=[8,12,13,14,9,10,11,1,1,0,2,3,4,5,6,7]
        for i,e in enumerate(useful_joints):
            output[0][i] = dict_pose[e]
        return output

In [5]:
data_reshaped= pickle_read("/content/drive/MyDrive/CVLAB_2022/data_reshaped.pickle")

In [6]:
movements = ['SQUAT', 'Lunges', 'Plank', 'Pick-up']
participants = ['Hugues', 'Sena', 'Vidit', 'Isinsu']

In [7]:
## Protocol : 
## Train on Hugues,Sena and Vidit
## Test on Isinsu

training_participants = ['Hugues', 'Sena', 'Vidit']
testing_participants = ['Isinsu']

In [8]:
_2d_poses_training = []
_3d_poses_training = []

_2d_poses_test = []
_3d_poses_test = []


## Training_Loop 
for exercice in data_reshaped.keys():
    for subject in training_participants:
        for variation in data_reshaped[exercice][subject].keys():
            for frame in data_reshaped[exercice][subject][variation].keys():
                for camera in ['6_1','6_2','6_3','6_4']:
                  _2d_poses_training.append(dictionary_to_2D_sample(data_reshaped[exercice][subject][variation][frame]['2D_gt'][camera]['p']))
                  #_3d_poses.append(dictionary_to_3D_sample(data_reshaped[exercice][subject][variation][frame]['3D_gt']['cam'][camera]))
                  _3d_poses_training.append(dictionary_to_3D_sample(data_reshaped[exercice][subject][variation][frame]['3D_gt']['world']))

with open(PROJECT_ROOT + "2D_CAMERA_TRAINING.pickle", 'wb') as f:
    pickle.dump(_2d_poses_training, f)

with open(PROJECT_ROOT + "3D_WORLD_TRAINING.pickle", 'wb') as f:
    pickle.dump(_3d_poses_training, f)

## Test_Loop 
for exercice in data_reshaped.keys():
    for subject in testing_participants:
        for variation in data_reshaped[exercice][subject].keys():
            for frame in data_reshaped[exercice][subject][variation].keys():
                for camera in ['6_1','6_2','6_3','6_4']:
                  _2d_poses_test.append(dictionary_to_2D_sample(data_reshaped[exercice][subject][variation][frame]['2D_gt'][camera]['p']))
                  #_3d_poses.append(dictionary_to_3D_sample(data_reshaped[exercice][subject][variation][frame]['3D_gt']['cam'][camera]))
                  _3d_poses_test.append(dictionary_to_3D_sample(data_reshaped[exercice][subject][variation][frame]['3D_gt']['world']))

with open(PROJECT_ROOT + "2D_CAMERA_TEST.pickle", 'wb') as f:
    pickle.dump(_2d_poses_test, f)

with open(PROJECT_ROOT + "3D_WORLD_TEST.pickle", 'wb') as f:
    pickle.dump(_3d_poses_test, f)

In [9]:
data_reshaped_3D= pickle_read(PROJECT_ROOT+"data/3d_cam_coordinates.pickle")
data_reshaped_standard = pickle_read("/content/drive/MyDrive/CVLAB_2022/data_reshaped.pickle")

In [10]:
_2d_poses_training = []
_3d_poses_training = []

_2d_poses_test = []
_3d_poses_test = []


## Training_Loop 
for exercice in data_reshaped_3D.keys():
    for subject in training_participants:
        for variation in data_reshaped_3D[exercice][subject].keys():
            for frame in data_reshaped_3D[exercice][subject][variation].keys():
              if(len(data_reshaped_3D[exercice][subject][variation][frame].keys()) == 4):
                for camera in ['6_1','6_2','6_3','6_4']:
                  _2d_poses_training.append(dictionary_to_2D_sample(data_reshaped_standard[exercice][subject][variation][frame]['2D_gt'][camera]['p']))
                  _3d_poses_training.append(data_reshaped_3D[exercice][subject][variation][frame][camera].T)



with open(PROJECT_ROOT + "2D_CAMERA_TRAINING_FOR_3D_CAMERA.pickle", 'wb') as f:
    pickle.dump(_2d_poses_training, f)

with open(PROJECT_ROOT + "3D_CAMERA_TRAINING_VF.pickle", 'wb') as f:
    pickle.dump(_3d_poses_training, f)

## Test_Loop  
for exercice in data_reshaped_3D.keys():
    for subject in testing_participants:
        for variation in data_reshaped_3D[exercice][subject].keys():
            for frame in data_reshaped_3D[exercice][subject][variation].keys():
              if(len(data_reshaped_3D[exercice][subject][variation][frame].keys()) == 4):
                for camera in ['6_1','6_2','6_3','6_4']:
                  _2d_poses_test.append(dictionary_to_2D_sample(data_reshaped_standard[exercice][subject][variation][frame]['2D_gt'][camera]['p']))
                  _3d_poses_test.append(data_reshaped_3D[exercice][subject][variation][frame][camera].T)



with open(PROJECT_ROOT + "2D_CAMERA_TEST_FOR_3D_CAMERA.pickle", 'wb') as f:
    pickle.dump(_2d_poses_test, f)

with open(PROJECT_ROOT + "3D_CAMERA_TEST_VF.pickle", 'wb') as f:
    pickle.dump(_3d_poses_test, f)